In [1]:
import os
import dask.dataframe as dd
import pandas as pd
os.chdir("/home/admin123/recsys/retail_rocket")
fp = "events.csv"
df = dd.read_csv(fp)
req_cols = ["visitorid", "itemid", "event", "timestamp"]
df = dd.from_pandas(df[req_cols].compute(), chunksize = int(1e6))


In [2]:
df = df.sample(frac = 0.005, replace = False, random_state = 1234).compute()

In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
df = dd.from_pandas(df, chunksize = 20000)

In [5]:
type(df)

dask.dataframe.core.DataFrame

In [6]:
df["timestamp"] = df["timestamp"].astype('datetime64[ns]')

In [7]:
df.head()

,visitorid,itemid,event,timestamp
21,522700,402970,view,1970-01-01 00:23:52.315488932
50,623675,26839,view,1970-01-01 00:23:52.314651255
73,69024,345963,view,1970-01-01 00:23:52.313481244
98,901571,458588,view,1970-01-01 00:23:53.222949983
143,75249,283146,view,1970-01-01 00:23:53.222790079


In [11]:
def create_ratings(userdf):
    userdf = userdf.reset_index()
    activities = userdf["event"]
    tc = len(activities)

    return tc

In [12]:
%time res = df.groupby(["visitorid", "itemid"]).apply(create_ratings).compute()


/home/admin123/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  """Entry point for launching an IPython kernel.


CPU times: user 9.16 s, sys: 40 ms, total: 9.2 s
Wall time: 9.22 s


In [13]:

max(res)

2

In [15]:
import pandas as pd
fp = "user_item_ratings.csv"
dfuir = pd.DataFrame(res)
dfuir.columns = ["ratings"]

In [16]:
dfuir.to_csv(fp, index = True)

In [17]:
del dfuir

In [37]:
si_series = df["visitorid"].value_counts().compute()

In [38]:
si_series.name ="site_interactions"

In [47]:
fp = "user_features.csv"
dfup = si_series.to_frame()
dfup = dfup.reset_index()
dfup.columns = ["visitorid", "site_interactions"]
dfup.to_csv(fp, index = False)

In [46]:
dfup.head()

,visitorid,site_interactions
0,1150086,40
1,530559,24
2,152963,17
3,371606,15
4,247235,12


In [49]:
si_series.max()

40